<a href="https://colab.research.google.com/github/wlghksx/Deep-Learning-from-Scratch/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0_%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94_%EB%94%A5%EB%9F%AC%EB%8B%9D_1_Chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5 오차역전파법(Backpropagation)

앞 장에서는 신경망 학습에 대해서 설명. 그때 신경망의 가중치 매개변수의 기울기는 수치 미분을 사용하여 구했다. 수치 미분은 단순하고 구현하기도 쉽지만 시간이 오래걸린다는게 단점이다. 이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 '오차역전파법(Backpropagation)'을 배워보겠다.

이 책에서는 수식이 아닌 좀 더 직관적인 계싼 그래프를 통해 시각적으로 이해해보도록 하자

## 5.1 계산 그래프

: 계산 과정을 그래프로 나타낸 것. 노드와 엣지를 가지고 표현.

: 계산을 왼쪽에서 오른쪽으로 진행하는 것을 순전파 (Forward propagation)

: 반대로 진행하는 것을 역전파(Back propagation)

## 5.1.2 국소적 계산

: 국소적 계산이란 '자신과 직접 관계된 작은 범위' 라는 뜻.

: 결국 계산 그래프는 전체 계산이 아무리 복잡하더라도 각 단계에서 하는 일은 해당 노드의 국소적 계산이다. 따라서 계산은 단순하지만, 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 할 수 있다.

## 5.1.3 왜 계산 그래프로 푸는가?

실제 계산 그래프를 사용하는 가장 큰 이유는 역전파를 통해 '미분'을 효울적으로 계산할 수 있는 점에 있다.

계산그래프에서의 거꾸로 화살표가 결국 미분값에 해당한다. 즉 변수의 변화에 따라 결과가 얼마나 달라지는지를 나타내주는 값이 된다.

## 5.2 연쇄법칙

즉, 후의 노드의 신호에 국소적 미분을 곱한 값을 이전 노드로 다시 전달하는 것을 역전파라 하는데, 이러한 방식을 따르면 목표로 하는 미분 값을 효율적으로 구할 수 있다.

이러한 역전파를 연쇄법칙의 원리로 설명할 수 있다.

## 5.3 역전파

## 5.3.1 덧셈 노드의 역전파

덧셈 노드에서의 역전파는 결국 결과적으로 입력된 값을 그대로 다음노드로 보내게 된다.

## 5.3.2 곱셈 노드의 역전파

곱셈 노드에서의 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값을'곱하여 하류로 보낸다.

# 5.4 단순한 계층 구하기

## 5.4.1 곱셈 계층

In [1]:
class MulLayer :
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self,x,y):
    self.x = x
    self.y = y
    out = x*y
    
    return out
  
  def backward(self,dout):
    dx = dout * self.y # x와 y를 바꾼다
    dy = dout * self.x

    return dx, dy

In [ ]:
# coding: utf-8
from layer_naive import *


apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)


# 5.5 활성화 함수 계층 구현하기

## 5.5.1 ReLU 계층

In [4]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx



순전파 때의 입력 값이 0 이하면 역전파 때의 값은 0이 돼야 한다. 그래서 역전파 때는 순전파 때 만들어 둔 mask를 써서 mask 원소가 True인 곳에는 상류에서 전파된 dout를 0으로 설정한다

## 5.5.2 Sigmoid 계층

In [5]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


# 5.6 Affine/Softmax 계층 구현하기

## 5.6.1 Affine 계층

신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서는 '어파인 변환'이라고 한다. 그래서 이 책에서는 어파인 변환을 수행하는 처리를 'Affine 계층'이라는 이름으로 구현한다.

In [6]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


## 5.6.1 Softmax - with - Loss 계층

마지막으로 출력층에서 사용하는 소프트맥스 함수에 관해 설명하겠다.

앞에서 말했듯이 소프트맥스 함수는 입력 값을 정규화(출력의 합이 1이 되도록)하여 출력한다.

손글씨 숫자는 가짓수가 10개(10클래스 분류)이므로 Softmax계층의 입력은 10개가 된다.

신경망에서 주로 수행하는 작업은 학습과 추론이다. 추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다. 예컨대 신경망은 추론할 떄는 마지막 Affine 계층의 출력을 인식 결과로 이용한다. 또한 신경망에서 정규화하지 않는 출력 결과에서는 Softmax 앞의 Affine 계를 점수라고 한다. 

즉, 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되나 Softmax 계층은 필요 없다는 것이다. 

하지만 신경망을 학습할 때는 Softmax 계층이 필요하다.

이제 소프트맥스 계츠을 구현할 텐데, 교차 엔트로피 오차도 포함하여 'Softmax - with - Loss 계층'이라는 이름으로 구현한다. 먼저 Softmax- with-Loss 계층의 계산 그래프를 살펴보자.

소프트맥스의 손실함수로 '교차 엔트로피 오차'를 사용하면 softmax 계층의 역전파가 (y1-t1, y2-t2, y3-t3) 로 말끔히 떨어진다

또한 회귀의 출력층에서 사용하는 항등함수의 손실함수로 오차제곱합을 사용하는 이유도 이와 같다. 즉, 항등함수의 손실함수로 오차제곱합을 사용하면 역전파의 결과가 (y1-t1,y2-t2,y3-t3)로 말끔히 떨어진다.

예를 들어, 정답 레이블이 (0,1,0) 일 떄, Softmax 계층이 (0.3,0.2,0.5)를 출력했다고 한다면,

정답의 인덱스는 1인데 이때의 확률은 겨우 20%라서 이 시점의 신경망은 제대로 인식하지 못하고 있는 것이다. 따라서 결과적으로 역전파는 (0.3, -0.8, 0.5)라는 커다란 오차를 전파하게 된다.

결과적으로 Softmax 앞 계층들은 그 큰 오차로부터 큰 깨달음을 얻게 된다.

In [7]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx


# 5.7 오차역전파법 구하기

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


## 5.7.3 오차역전파법으로 구한 기울기 검증하기

결국 지금까지 기울기를 구하는 방법을 2가지를 설명했다.
1. 수치 미분을 써서 구하는 방법
2. 해석적으로 수식을 풀어 구하는 방법

=> 후자인 해석적 방법은 오차역전파법을 이용하여 매개변수가 많아도 효율적으로 계산할 수 있었다. 그러니 이제부터는 느린 수치 미분 대신 오차역전파법을 사용하도록 하자.

- 하지만 수치미분이 아무른 필요가 없는 것은 아니다. 수치 미분의 이점은 구현하기가 쉽다는 점이다. 수치미분의 구현에는 버그가 숨어있기 어려운 반면, 오차역전파법은 복잡해서 종종 실수를 하곤 한다. 그래서 수치 미분의 결과와 오차역전파법의 결과를 비교하여 제대로 구현했는지 검증할 수 있다.

- 이렇게 두 방식으로 구한 기울기가 동일함을 확인하는 작업을 '기울기 확인'이라고 한다.